# Welcome to Modal notebooks!

Write Python code and collaborate in real time. Your code runs in Modal's
**serverless cloud**, and anyone in the same workspace can join.

This notebook comes with some common Python libraries installed. Run
cells with `Shift+Enter`.

In [1]:
# ===================== SAFE INSTALL (KAGGLE-COMPATIBLE) =====================
!pip install -U transformers tokenizers accelerate python-chess tqdm --quiet

# ===================== ENV =====================
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# ===================== IMPORTS =====================
import torch
import chess.pgn
import io
import random
import json
from tqdm import tqdm
from dataclasses import dataclass
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM, get_linear_schedule_with_warmup

print("Torch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# ===================== CONFIG =====================
@dataclass
class Config:
    base_model = "lazy-guy12/chess-llama"
    pgn_path = "/root/Anand.pgn"
    output_dir = "/root/anand_chess_modelv3"

    max_seq_length = 128
    batch_size = 5
    epochs = 15
    lr = 1e-5
    warmup_ratio = 0.1
    weight_decay = 0.01

    device = "cuda" if torch.cuda.is_available() else "cpu"

cfg = Config()

# ===================== LOAD PGN =====================
def load_anand_games(path):
    games = []
    with open(path, "r") as f:
        pgn_io = io.StringIO(f.read())

    while True:
        game = chess.pgn.read_game(pgn_io)
        if game is None:
            break

        h = game.headers
        if "Anand" not in h.get("White","") and "Anand" not in h.get("Black",""):
            continue

        moves = [m.uci() for m in game.mainline_moves()]
        if len(moves) < 5:
            continue

        games.append(f'[Result "{h.get("Result","*")}"] ' + " ".join(moves))

    return games

games = load_anand_games(cfg.pgn_path)
random.shuffle(games)

split = int(0.85 * len(games))
train_data = games[:split]
val_data = games[split:]

print("Games used:", len(games))

# ===================== DATASET =====================
class ChessDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.data[idx],
            max_length=cfg.max_seq_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        input_ids = enc["input_ids"].squeeze()
        attention_mask = enc["attention_mask"].squeeze()

        labels = input_ids.clone()
        labels[attention_mask == 0] = -100   # critical fix

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# ===================== TOKENIZER =====================
tokenizer = AutoTokenizer.from_pretrained(
    cfg.base_model,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ===================== MODEL (SAFE LOAD) =====================
model = AutoModelForCausalLM.from_pretrained(
    cfg.base_model,
    torch_dtype=torch.float32,
    trust_remote_code=True,
    device_map=None
)

model = model.to(cfg.device)
torch.cuda.empty_cache()

print("Model loaded successfully")

# ===================== LOADERS =====================
train_loader = DataLoader(
    ChessDataset(train_data, tokenizer),
    batch_size=cfg.batch_size,
    shuffle=True
)

val_loader = DataLoader(
    ChessDataset(val_data, tokenizer),
    batch_size=cfg.batch_size
)

optimizer = AdamW(
    model.parameters(),
    lr=cfg.lr,
    weight_decay=cfg.weight_decay
)

total_steps = len(train_loader) * cfg.epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    int(total_steps * cfg.warmup_ratio),
    total_steps
)

# ===================== TRAIN =====================
best_val = float("inf")

for epoch in range(cfg.epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Train"):
        batch = {k:v.to(cfg.device) for k,v in batch.items()}
        out = model(**batch)
        loss = out.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k:v.to(cfg.device) for k,v in batch.items()}
            val_loss += model(**batch).loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1} | Val Loss: {val_loss:.4f}")

    if val_loss < best_val:
        best_val = val_loss
        model.save_pretrained(cfg.output_dir)
        tokenizer.save_pretrained(cfg.output_dir)

# ===================== GENERATION (SAFE) =====================
def generate_move(moves):
    text = '[Result "*"] ' + " ".join(moves)
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k:v.to(cfg.device) for k,v in inputs.items() if k != "token_type_ids"}

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=6,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id
        )

    gen = tokenizer.decode(out[0], skip_special_tokens=True).split()
    for m in gen[len(text.split()):]:
        if len(m) == 4 and m.isalnum():
            return m
    return None

print("Test move:", generate_move(["e2e4","e7e5","g1f3","b8c6"]))





[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Torch: 2.8.0+cu129
CUDA: True
GPU: NVIDIA A100 80GB PCIe
Games used: 4177


tokenizer_config.json:   0%|          | 0.00/945 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

...

Epoch 14 Train: 100%|████████████████████████████████████████████████████████| 710/710 [00:29<00:00, 23.89it/s]


Epoch 14 | Val Loss: 1.5916


Epoch 15 Train: 100%|████████████████████████████████████████████████████████| 710/710 [00:29<00:00, 23.72it/s]


Epoch 15 | Val Loss: 1.5915
Test move: b5a4


IsADirectoryError: [Errno 21] Is a directory: '/root'